## References

https://stackoverflow.com/questions/69195950/problem-with-inputs-when-building-a-model-with-tfbertmodel-and-autotokenizer-fro

In [1]:
import json
from pathlib import Path
from typing import List

import tensorflow as tf
from tensorflow import keras
import keras.layers as L
import keras.models as M
import keras.losses as LL
import tensorflow_datasets as tfds
import tensorflow_models as tfm
from tensorflow_text.tools.wordpiece_vocab import bert_vocab_from_dataset

import transformers
from transformers import AutoTokenizer, TFDistilBertForTokenClassification
import tensorflow_hub as hub
import numpy as np
import pandas as pd

2023-01-08 13:55:23.282749: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-08 13:55:23.584253: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-01-08 13:55:23.584275: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-01-08 13:55:24.788303: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [2]:
(ds_train, ds_val, ds_test), info = tfds.load("conll2003", with_info=True, split=["train", "dev", "test"])

2023-01-08 13:55:28.289450: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/calcifer/git/marco/learn-deep-learning/.env/lib/python3.10/site-packages/cv2/../../lib64:
2023-01-08 13:55:28.289560: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-01-08 13:55:28.289654: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (calcifer-Inspiron-7370): /proc/driver/nvidia/version does not exist
2023-01-08 13:55:28.290311: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, reb

In [280]:
tag_names = info.features["ner"].names
tag_ids = range(len(tag_names))
label_ids_to_tags = dict(enumerate(tag_names))

In [4]:
sample = ds_train.as_numpy_iterator().next()
sample

2023-01-08 13:55:28.932458: W tensorflow/core/kernels/data/cache_dataset_ops.cc:856] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


{'chunks': array([ 0, 17, 11, 21, 22, 11, 12, 12, 12,  0,  3, 11, 12, 12, 21, 22, 22,
        11, 12, 12, 12,  0,  0, 21, 11, 12, 12, 12, 11, 21, 22, 22, 22,  0]),
 'ner': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0]),
 'pos': array([ 0, 15, 28, 41, 39, 15, 18, 11, 21,  6, 30, 29, 16, 24, 41, 30, 39,
        11, 10, 11, 21,  6,  0, 38, 11, 22, 24, 21, 44, 38, 35, 37, 40,  7]),
 'tokens': array([b'"', b'If', b'they', b"'re", b'saying', b'at', b'least', b'20',
        b'percent', b',', b'then', b'their', b'internal', b'forecasts',
        b'are', b'probably', b'saying', b'25', b'or', b'30', b'percent',
        b',', b'"', b'said', b'one', b'Sydney', b'media', b'analyst',
        b'who', b'declined', b'to', b'be', b'named', b'.'], dtype=object)}

## Prepare the dataset

In [38]:
VOCAB_FILE = Path.cwd() / "vocab.txt"

TRANSFORMER_MODEL_NAME = "distilbert-base-cased"

In [6]:
transformers_tokenizer = AutoTokenizer.from_pretrained(TRANSFORMER_MODEL_NAME)

In [21]:
df = (
    pd.DataFrame.from_dict(transformers_tokenizer.get_vocab(), orient="index")
    .reset_index()
    .rename({"index": "token", 0: "idx"}, axis=1)
    .sort_values("idx")
    .reset_index(drop=True)
)
df.head()

,token,idx
0,[PAD],0
1,[unused1],1
2,[unused2],2
3,[unused3],3
4,[unused4],4


In [39]:
df["token"].to_csv(path_or_buf=VOCAB_FILE, index=False, header=False)

## Create the tokenizer

In [96]:
tokenizer = tfm.nlp.layers.BertTokenizer(
    vocab_file=VOCAB_FILE.as_posix(),
    lower_case=False,
    tokenize_with_offsets=True,
)

In [97]:
sample = (
    ds_train.map(lambda r: r["tokens"])
    .take(1)
    .as_numpy_iterator().next()
)
sample

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089
2023-01-08 13:55:30.421244: W tensorflow/core/kernels/data/cache_dataset_ops.cc:856] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


array([b'"', b'If', b'they', b"'re", b'saying', b'at', b'least', b'20',
       b'percent', b',', b'then', b'their', b'internal', b'forecasts',
       b'are', b'probably', b'saying', b'25', b'or', b'30', b'percent',
       b',', b'"', b'said', b'one', b'Sydney', b'media', b'analyst',
       b'who', b'declined', b'to', b'be', b'named', b'.'], dtype=object)

In [213]:
tokens, offsets_start, offsets_end = tokenizer(sample)
np_tokens = tokens.merge_dims(0, 1).merge_dims(0, 1).numpy()
pprint(np_tokens)
pprint(tokens)

x=array([  100,  1409,  1152,   112,  1231,  2157,  1120,  1655,  1406,
        3029,   100,  1173,  1147,  4422, 24647,  1116,  1132,  1930,
        2157,  1512,  1137,  1476,  3029,   100,   100,  1163,  1141,
        3122,  2394, 14582,  1150,  5799,  1106,  1129,  1417,   119],
      dtype=int32)
x=<tf.RaggedTensor [[[100]], [[1409]], [[1152]], [[112], [1231]], [[2157]], [[1120]],
 [[1655]], [[1406]], [[3029]], [[100]], [[1173]], [[1147]], [[4422]],
 [[24647, 1116]], [[1132]], [[1930]], [[2157]], [[1512]], [[1137]],
 [[1476]], [[3029]], [[100]], [[100]], [[1163]], [[1141]], [[3122]],
 [[2394]], [[14582]], [[1150]], [[5799]], [[1106]], [[1129]], [[1417]],
 [[119]]]>


In [232]:
tokens.merge_dims(0, 2)

<tf.Tensor: shape=(36,), dtype=int32, numpy=
array([  100,  1409,  1152,   112,  1231,  2157,  1120,  1655,  1406,
        3029,   100,  1173,  1147,  4422, 24647,  1116,  1132,  1930,
        2157,  1512,  1137,  1476,  3029,   100,   100,  1163,  1141,
        3122,  2394, 14582,  1150,  5799,  1106,  1129,  1417,   119],
      dtype=int32)>

In [236]:
tokens.row_lengths()

<tf.Tensor: shape=(34,), dtype=int64, numpy=
array([1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])>

In [235]:
tokens.merge_dims(0, 1).row_lengths()

<tf.Tensor: shape=(35,), dtype=int64, numpy=
array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])>

Check that the vocab porting was successful

In [188]:
transformers_tokenizer.convert_ids_to_tokens(np_tokens)

['[UNK]',
 'If',
 'they',
 "'",
 're',
 'saying',
 'at',
 'least',
 '20',
 'percent',
 '[UNK]',
 'then',
 'their',
 'internal',
 'forecast',
 '##s',
 'are',
 'probably',
 'saying',
 '25',
 'or',
 '30',
 'percent',
 '[UNK]',
 '[UNK]',
 'said',
 'one',
 'Sydney',
 'media',
 'analyst',
 'who',
 'declined',
 'to',
 'be',
 'named',
 '.']

## Prepare the dataset

In [209]:
def pprint(x):
    print(f"{x=}")

In [288]:
def tokenize(r):
    tokens, offsets_start, offsets_end = tokenizer(r["tokens"])
    tokens_flat_0 = tokens.merge_dims(0, 1)
    tokens_flat = tokens_flat_0.merge_dims(0, 1)
    labels_aligned_0 = tf.repeat(r["ner"], tokens.row_lengths())
    labels_aligned = tf.repeat(labels_aligned_0, tokens_flat_0.row_lengths())
    return tokens_flat, labels_aligned

In [291]:
MAX_LENGTH = 18
for tokens, labels in ds_train.map(tokenize).shuffle(10).take(5):
    bert_words = transformers_tokenizer.convert_ids_to_tokens(tokens)
    print(*[f"{x:<12}" for x in bert_words][:MAX_LENGTH])
    print(*[f"{label_ids_to_tags[x]:<12}" for x in labels.numpy()][:MAX_LENGTH])
    print("\n")

Thailand     '            s            powerful     military     thinks       the          government   is           dish         ##ones       ##t          and          Prime        Minister     Ban          ##har        ##n         
B-LOC        O            O            O            O            O            O            O            O            O            O            O            O            O            O            B-PER        B-PER        B-PER       


L            ##OS         AN           ##GE         ##LE         ##S          70           60           .            53           ##8          1           
B-ORG        B-ORG        I-ORG        I-ORG        I-ORG        I-ORG        O            O            O            O            O            O           


Geelong      21           13           1            7            228          ##8          1940         117          .            9            54          
B-ORG        O            O            O            O       

2023-01-08 15:27:16.125914: W tensorflow/core/kernels/data/cache_dataset_ops.cc:856] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


In [5]:
BATCH_SIZE = 64
BUFFER_SIZE = 100

In [53]:
ds_train_prep = (
    ds_train.map(lambda r: (r["tokens"], r["ner"]))
    .shuffle(buffer_size=BUFFER_SIZE)
    .padded_batch(BATCH_SIZE, padded_shapes=([None], [None]))
)
ds_val_prep = (
    ds_val.map(lambda r: (r["tokens"], r["ner"]))
    .padded_batch(BATCH_SIZE, padded_shapes=([None], [None]))
)

In [29]:
sample = ds_train_prep.as_numpy_iterator().next()
sample[0].shape, sample[1].shape

2023-01-08 00:29:38.334500: W tensorflow/core/kernels/data/cache_dataset_ops.cc:856] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


((64, 52), (64, 52))

In [30]:
sample = ds_train_prep.map(lambda tokens, ner: tokens).as_numpy_iterator().next()

2023-01-08 00:29:39.577616: W tensorflow/core/kernels/data/cache_dataset_ops.cc:856] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


## Model setup

### Setup the tokenizer

In [31]:
TRANSFORMER_MODEL_NAME = "distilbert-base-cased"

In [57]:
transformers_tokenizer = AutoTokenizer.from_pretrained(TRANSFORMER_MODEL_NAME)

In [58]:
distilbert = TFDistilBertForTokenClassification.from_pretrained(TRANSFORMER_MODEL_NAME, output_hidden_states=True, output_attentions=True)

Some layers from the model checkpoint at distilbert-base-cased were not used when initializing TFDistilBertForTokenClassification: ['vocab_layer_norm', 'vocab_projector', 'vocab_transform', 'activation_13']
- This IS expected if you are initializing TFDistilBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['dropout_59', 'classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Define the model

In [59]:
def tokenize(tensor):
    arr = tensor.numpy().astype(str)
    tokenized = transformers_tokenizer(arr.tolist(), return_tensors="tf", is_split_into_words=True, padding="longest")
    return tokenized['input_ids'], tokenized['attention_mask']

In [62]:
inputs = L.Input(shape=(None,), dtype=tf.string)
input_ids, attention_mask = tf.py_function(tokenize, inp=[inputs], Tout=(tf.int32, tf.int32))
input_ids = tf.ensure_shape(input_ids, (None, inputs.shape[0]))
attention_mask = tf.ensure_shape(attention_mask, (None, inputs.shape[0]))
x = distilbert(input_ids, attention_mask)["logits"]
x = L.Dense(units=len(tag_names), activation="softmax")(x)
model = M.Model(inputs=inputs, outputs=x)

In [63]:
model.summary(expand_nested=True, show_trainable=True)

Model: "model_3"
_____________________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     Trainable  
 input_6 (InputLayer)           [(None, None)]       0           []                               Y          
                                                                                                             
 tf.py_function_5 (TFOpLambda)  [None, None]         0           ['input_6[0][0]']                Y          
                                                                                                             
 tf.ensure_shape_6 (TFOpLambda)  (None, None)        0           ['tf.py_function_5[0][0]']       Y          
                                                                                                             
 tf.ensure_shape_7 (TFOpLambda)  (None, None)        0           ['tf.py_function_5[0][1]']       Y    

In [75]:
arr, tags = ds_train_prep.as_numpy_iterator().next()
arr = arr.astype(str)
print(f"{arr}")
print(f"{arr.shape}")

[['SK' 'Ceske' 'Budejovice' ... '' '' '']
 ['The' 'following' 'table' ... '' '' '']
 ['"' 'But' 'the' ... '' '' '']
 ...
 ['HORSE' 'RACING' '-' ... '' '' '']
 ['Results' 'of' 'Major' ... '' '' '']
 ['Africa' ')' '66' ... '' '' '']]
(64, 52)


2023-01-08 00:37:14.943847: W tensorflow/core/kernels/data/cache_dataset_ops.cc:856] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


In [77]:
tags.shape

(64, 52)

In [65]:
tokenized = transformers_tokenizer(arr.tolist(), return_tensors="tf", is_split_into_words=True, padding=True)
tokenized.keys()

dict_keys(['input_ids', 'attention_mask'])

In [66]:
output = model.predict(arr)

2/2 [==============================] - 3s 725ms/step


In [67]:
output.to_tensor().shape

TensorShape([64, 66, 9])

## Training

In [79]:
LL.SparseCategoricalCrossentropy()(tags, output)

ValueError: `labels.shape` must equal `logits.shape` except for the last dimension. Received: labels.shape=(64, 52) and logits.shape=(64, 66, 9)

In [45]:
EPOCHS = 10

In [54]:
model.compile(
    optimizer="adam",
    loss=LL.SparseCategoricalCrossentropy(),
    metrics=[
        keras.metrics.Accuracy(), 
        keras.metrics.TrueNegatives(), 
        keras.metrics.TruePositives(),
    ],
)

In [55]:
history = model.fit(
    ds_train_prep,
    validation_data=ds_val_prep,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
)

Epoch 1/10


ValueError: in user code:

    File "/home/calcifer/git/marco/learn-deep-learning/.env/lib/python3.10/site-packages/keras/engine/training.py", line 1249, in train_function  *
        return step_function(self, iterator)
    File "/home/calcifer/git/marco/learn-deep-learning/.env/lib/python3.10/site-packages/keras/engine/training.py", line 1233, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/calcifer/git/marco/learn-deep-learning/.env/lib/python3.10/site-packages/keras/engine/training.py", line 1222, in run_step  **
        outputs = model.train_step(data)
    File "/home/calcifer/git/marco/learn-deep-learning/.env/lib/python3.10/site-packages/keras/engine/training.py", line 1028, in train_step
        return self.compute_metrics(x, y, y_pred, sample_weight)
    File "/home/calcifer/git/marco/learn-deep-learning/.env/lib/python3.10/site-packages/keras/engine/training.py", line 1122, in compute_metrics
        self.compiled_me

In [ ]:
tfm.nlp.models.BertTokenClassifier